In [1]:
from google.colab import drive
drive.mount('/content/drive')

import argparse
import csv
import os
import sys
import re
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.stats import loguniform
from scipy.io import loadmat
from sklearn import metrics
from torch.utils import data

sys.path.append('/content/drive/MyDrive/Tesi_Magistrale')

from Data_Processing_Utils import Norm_each_sub_by_own_param,NormalizeData, train_val_test_split_df, PlotLoss, plot_confusion_matrix, \
    Get_Sub_Norm_params, windowing_Dataframe

from DATALOADERS import dataframe_dataset_triplet

from MODELS import MKCNN, MKCNN_grid, random_choice, train_model_triplet, pre_train_model_triplet, train_model_standard, MultiKernelConv2D_grid
pd.options.mode.chained_assignment = None

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# custom pandas dataset class modified to provide also the angle
class Pandas_Dataset(data.Dataset):

    def __init__(self, df_grouped_by_samples, return_sub=False):
        self.grouped = df_grouped_by_samples
        self.channels = [i for i in df_grouped_by_samples.obj.columns if 'ch' in i]
        self.indices = list(df_grouped_by_samples.indices)
        self.return_sub = return_sub

    def __len__(self):
        return len(self.grouped)

    def __getitem__(self, index):
        picked_smp = self.grouped.get_group(self.indices[index])
        # Picking only the channels columns from a single sample
        sample = torch.tensor(picked_smp.loc[:, self.channels].values).type(torch.float32)
        # picking only one label for each sample
        label = torch.tensor(picked_smp.loc[:, ['label']].head(1).values[0][0]).type(torch.int8)
        angle = torch.tensor(picked_smp.loc[:, ['angle']].head(1).values[0][0]).type(torch.int16)
        if self.return_sub:
            # picking only one subject
            sub = torch.tensor(picked_smp.loc[:, ['sub']].head(1).values[0][0]).type(torch.int8)
            return sample, label, angle, sub
        else:

            # It's missing the part in which I Normalize the data for each subject, or for each subject and channels
            # It's not a good idea to do that on the dataloader while producing results
            return sample, label, angle


In [3]:
file_path = '/content/drive/MyDrive/Tesi_Magistrale'
data_path = '/content/drive/MyDrive/Tesi_Magistrale/DATA'
subj = 'Sub3'
test_rep = 3
WND_LEN = 200
NORM_TYPE = 'zero_to_one'
day = 1
single_model = True

# NOTE 1: the single model test set works also for the multi model
# NOTE 2: No need to norm test set

database = data_path + f'/{subj}_offset_relabel/day{day}/Dataframe/'
save_path = data_path + f'/{subj}_offset_relabel/day{day}/test_rep{test_rep}/Cross_rep/Testing_Results/'
weight_path = data_path + f'/{subj}_offset_relabel/day{day}/test_rep{test_rep}/Cross_rep/Best_States/'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [4]:
print('Available weights:')
for w in os.listdir(weight_path):
  print(w)
print('\nAvailable sets')
for s in os.listdir(database):
  print(s)

Available weights:
state_dict_wnd_200_zero_to_one_sub_rect_True_ang_1_day1.pth
state_dict_wnd_200_zero_to_one_sub_rect_True_ang_2_day1.pth
state_dict_wnd_200_zero_to_one_sub_rect_True_ang_3_day1.pth
state_dict_wnd_200_zero_to_one_sub_rect_True_ang_4_day1.pth
state_dict_wnd_200_zero_to_one_sub_rect_True_ang_5_day1.pth
state_dict_wnd_200_zero_to_one_sub_rect_True_single_model_day1.pth

Available sets
dataframe_wnd_200_angle_merged.csv


In [5]:
df = pd.read_csv(database + f'dataframe_wnd_{WND_LEN}_angle_merged.csv')
df = df[df['rep'] == test_rep]
df.drop('angle_index', axis=1, inplace = True)

NORM_MODE = 'sub'
NORM_TYPE = 'zero_to_one'
RECT = True

df = Norm_each_sub_by_own_param(df, mode= NORM_MODE, norm_type= NORM_TYPE, rectify=RECT )
if 'Unnamed: 0' in df.columns:
    df.drop('Unnamed:0', inplace= True)

minAngle, maxAngle = df['angle'].abs().min(), df['angle'].abs().max()
print(f"minAngle: {minAngle} maxAngle: {maxAngle}")
angleRange = maxAngle - minAngle
interval = 5*round(angleRange * 0.125/5) #round interval to the closest multiple of 5
residue = angleRange-interval*8
intervals = [abs(minAngle+interval), abs(minAngle+interval*3), abs(maxAngle-interval*3), abs(maxAngle-interval)]
print(f"Angle Intervals: {intervals}")

df = Pandas_Dataset(df.groupby('sample_index'))
batch_size = 1
num_workers = 0
params = {'batch_size': batch_size,
            'shuffle': False,
            # 'sampler': sampler,
            'num_workers': num_workers,
            'drop_last': False}
df_dataloader = data.DataLoader(df, **params)

print('Dataset Ready')

RECTIFICATION OF THE DATAFRAME
DONE
Norm SUB: 1
DONE
minAngle: 0 maxAngle: 115
Angle Intervals: [15, 45, 70, 100]
Dataset Ready


In [6]:
kernels_gap = [g for g in range(0, 3 * round(WND_LEN / 20), round(WND_LEN / 20))]
kernel_sizes = np.full((3, 5, 2), [1, 3])
for j in range(3):
    for i in range(5):
        kernel_sizes[j][i][0] = (round(WND_LEN / 20) * (i + 1) + kernels_gap[j])

n_classes = 10
net = {'N_multik': 32, 'N_Conv_conc': 64, 'N_SepConv': 128,
            'Kernel_multi_dim': kernel_sizes[0],
            'Kernel_Conv_conc': 1,
            'act_func': nn.ReLU(),
            'Pool_Type': nn.MaxPool2d,
            'wnd_len':WND_LEN
            }

if single_model:
    w = torch.load(weight_path+f'state_dict_wnd_{WND_LEN}_zero_to_one_sub_rect_True_single_model_day{day}.pth', map_location = torch.device(device))
    print('Weights loaded')
    m = MKCNN_grid(net, num_classes=n_classes)
    m.to(device)
    softmax_block = nn.Softmax(dim=1)
    print('Model built')
    m.load_state_dict(w)
    print('Model Ready')
else:
    ws = []
    for ANGLE in range(1,6):
        ws.append(torch.load(weight_path+f'state_dict_wnd_{WND_LEN}_zero_to_one_sub_rect_True_ang_{ANGLE}_day{day}.pth', map_location = torch.device(device)))
    print('Weights Loaded')
    m1 = MKCNN_grid(net, num_classes=n_classes).to(device)
    m2 = MKCNN_grid(net, num_classes=n_classes).to(device)
    m3 = MKCNN_grid(net, num_classes=n_classes).to(device)
    m4 = MKCNN_grid(net, num_classes=n_classes).to(device)
    m5 = MKCNN_grid(net, num_classes=n_classes).to(device)
    softmax_block = nn.Softmax(dim=1)
    print('Models built')
    m1.load_state_dict(ws[0])
    m2.load_state_dict(ws[1])
    m3.load_state_dict(ws[2])
    m4.load_state_dict(ws[3])
    m5.load_state_dict(ws[4])
    print("Models ready")

Weights loaded
Model built
Model Ready


In [7]:
if single_model:
    m.eval()
else:
    m1.eval()
    m2.eval()
    m3.eval()
    m4.eval()
    m5.eval()
print("Lock 'n loaded. Ready to rouble")

with torch.no_grad():
    y_pred = []
    y_true = []
    for inputs, labels, angle in df_dataloader:
        inputs = inputs[:, None, :, :]
        inputs = inputs.to(device)
        angle = abs(angle.cpu().data.numpy()[0])
        labels_np = labels.cpu().data.numpy()
        if single_model:
            output, _ = m.forward(inputs)
            model = 'single'
        else:
            if angle<intervals[0]:
                output, _ = m1.forward(inputs)
                model = 1
            elif angle<intervals[1]:
                output, _ = m2.forward(inputs)
                model = 2
            elif angle<intervals[2]:
                output, _ = m3.forward(inputs)
                model = 3
            elif angle<intervals[3]:
                output, _ = m4.forward(inputs)
                model = 4
            else:
                output, _ = m5.forward(inputs)
                model = 5
        outputs_np = softmax_block(output)
        outputs_np = outputs_np.cpu().data.numpy()
        outputs_np = np.argmax(outputs_np, axis=1)
        y_pred = np.append(y_pred, outputs_np)
        y_true = np.append(y_true, labels_np)
        print(f"Model: {model}\tAngle: {angle}\tPredicted pose: {outputs_np}\tReal Pose: {labels_np}")

    print("Testing Done!")


print("Saving Confusion Matrix")
from Data_Processing_Utils import  plot_confusion_matrix

poses = ['medium_wrap', 'lateral','power_sphere', 'power_disk', 'prismatic_pinch',
            'index_extension', 'wave_out', 'wave_in', 'fist', 'open_hand']

cm = metrics.confusion_matrix(y_true, y_pred)
title = f"day{day} {'single model' if single_model else 'multi model'} rep {test_rep}" # title on the confusion matrix
if not os.path.exists(save_path):
    os.makedirs(save_path)
plot_confusion_matrix(cm, target_names=poses, title=f'Confusion Matrix {title} {NORM_TYPE}',
                                path_to_save=save_path + f'cm_day{day}_{NORM_TYPE}_{"single_model" if single_model else "multi_model"}.png')


Lock 'n loaded. Ready to rouble
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [0]	Real Pose: [0]
Model: single	Angle: 0	Predicted pose: [